In [1]:
import torch
import gym
from gym import envs
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.distributions import Categorical
from torchvision import transforms
import pickle

In [2]:
lenobs = 21190
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.l1 = nn.Linear(lenobs,25)
        self.l2 = nn.Linear(25,50)
        self.actor_lin1 = nn.Linear(50,3)
        self.l3 = nn.Linear(50,25)
        self.critic_lin1 = nn.Linear(25,1)

    def forward(self,x):
        x = F.normalize(x,dim=0)
        y = F.relu(self.l1(x))
        y = F.normalize(y,dim=0)
        y = F.relu(self.l2(y))
        y = F.normalize(y,dim=0)
#         actor = F.softmax(self.actor_lin1(y),dim=0)
        actor = F.log_softmax(self.actor_lin1(y),dim=0)
        c = F.relu(self.l3(y.detach()))
        critic = torch.tanh(self.critic_lin1(c))
        return actor, critic

In [3]:
env = gym.make('Pong-v0')
moveMapping = {
    0:0,
    1:2,
    2:3
}

In [28]:
import time

def play(model):
    score = 0.0
    observation = env.reset()
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
    ])
    
    model.eval()


    done = False
    a = time.time()
    while done == False:
        pobservation = torch.from_numpy(observation).permute(2,0,1)
        cropped_image = transforms.functional.crop(preprocess(pobservation),32,15,163,130)
        gs_image = transforms.functional.to_grayscale(cropped_image)
        gs_tensor = transforms.ToTensor()(gs_image)
        flattened_pobservation = gs_tensor.view(-1).float()
        policy, value = model(flattened_pobservation)
        sampler = Categorical(policy)
        action = sampler.sample()
        observation, reward, done, log = env.step(moveMapping[action.item()])
        if reward == 1.0:
            score+=1.0
            
#         env.render()
    
#     if done:
#         env.close()

    return score
        
        


In [46]:
import matplotlib.pyplot as plt
%matplotlib inline

score_distribution = [0 for _ in range(22)]
nog = 100 #number of games


# Loading saved model
model = ActorCritic()
model_path = './a2c_seq.pth'
m1 = torch.load(model_path)
model.load_state_dict(m1)


for _ in range(nog):
    score_distribution[int(play(model))]+=1
    
score_distribution

[77, 16, 4, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [45]:
# model_path_seq = './a2c_seq.pth'
# state_dict = model.state_dict()
# torch.save(state_dict, model_path_seq)

In [44]:
score_distribution

[81, 12, 3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Baseline

In [40]:
score_distribution

[65, 28, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Number of times agent won

In [11]:
one_reward_count = pickle.load(open('one_reward_count.pkl','rb'))
one_reward_count

[0]